<a href="https://colab.research.google.com/github/jtapiav/github-slideshow/blob/master/Extending_Kernighan_Lin_Partitioning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Bibliotecas

In [ ]:
import random

#Original Kernighan-Lin Partition

## Graph

In [ ]:
class CoreGraph:
    def __init__(self, cores, edges):
        self.cores = cores
        self.edges = edges

## Calculate Cost

In [ ]:
def calculate_cost(graph, partition1, partition2):
    cost = 0
    for edge in graph.edges:
        if (edge[0] in partition1 and edge[1] in partition2) or (edge[0] in partition2 and edge[1] in partition1):
            cost += 1
    return cost

## Swap Core

In [ ]:
def get_next_move(graph, partition1, partition2):
    best_cost = calculate_cost( graph, partition1, partition2 )
    best_move = None
    for ci in partition1:
        for cj in partition2:
            current_cost = calculate_cost(graph, partition1 - {ci} | {cj}, partition2 - {cj} | {ci})
            if current_cost < best_cost:
                best_cost = current_cost
                best_move = (ci, cj)
    return best_move

## Kernighan_Lin Partition

In [ ]:
def kernighan_lin(graph, partition1, partition2, level=0):
    best_partition1, best_partition2 = partition1.copy(), partition2.copy()
    current_partition1, current_partition2 = partition1.copy(), partition2.copy()

    max_iterations = 50

    for iteration in range(max_iterations):
        next_move = get_next_move(graph, current_partition1, current_partition2)
        if next_move is None:
            break  # Salimos del ciclo si no encontramos una mejor jugada
        ci, cj = next_move
        current_partition1.remove(ci)
        current_partition2.remove(cj)
        current_partition1.add(cj)
        current_partition2.add(ci)

        if calculate_cost(graph, current_partition1, current_partition2) < calculate_cost(graph, best_partition1, best_partition2):
            best_partition1, best_partition2 = current_partition1.copy(), current_partition2.copy()

    if len(best_partition1) == len(partition1) and len(best_partition2) == len(partition2):
        return best_partition1, best_partition2

    p1, p2 = best_partition1, best_partition2
    return p1, p2

## Extend Kernighan-Lin Partition

In [ ]:
def kl_partitioning(graph, level, cores_to_be_partitioned):
  cores_to_be_partitioned = set(cores_to_be_partitioned)  # Convert to set to allow set operations

  if len(cores_to_be_partitioned) < 4:
    return cores_to_be_partitioned

  cores_list = list(cores_to_be_partitioned)

  partition1 = set(random.sample(cores_list, len(cores_list) // 2))
  partition2 = cores_to_be_partitioned - partition1

  indent = " " * (level * 4)
  print(indent + f"Level {level}:")
  print(indent + f"Complete Set: {cores_to_be_partitioned}")
  print(indent + f"Start Partitions" + f" Cost: {calculate_cost(graph, partition1, partition2)}")
  print(indent + f" Partition 1: {set(partition1)}")
  print(indent + f" Partition 2: {set(partition2)}")

  partition1, partition2 = kernighan_lin(graph, partition1, partition2)

  indent = " " * (level * 4)
  print(indent + f"Final Partitions" + f" Cost: {calculate_cost(graph, partition1, partition2)}")
  print(indent + f" Partition 1: {set(partition1)}")
  print(indent + f" Partition 2: {set(partition2)}")

  partition1_1, partition1_2 = kl_partitioning(graph, level + 1, partition1)
  partition2_1, partition2_2 = kl_partitioning(graph, level + 1, partition2)

  partition1 = (partition1_1, partition1_2)
  partition2 = (partition2_1, partition2_2)

  return partition1, partition2


## Test

In [ ]:
# Ejemplo de uso
cores = {'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'C10', 'C11', 'C12', 'C13', 'C14', 'C15', 'C16'}
edges = {('C1', 'C2'), ('C1', 'C3'), ('C2', 'C4'), ('C3', 'C4'), ('C3', 'C5'), ('C4', 'C6'), ('C5', 'C6'),
         ('C7', 'C8'), ('C7', 'C9'), ('C8', 'C10'), ('C9', 'C10'), ('C9', 'C11'), ('C10', 'C12'), ('C11', 'C12'),
         ('C13', 'C14'), ('C13', 'C15'), ('C14', 'C16'), ('C15', 'C16')}
graph = CoreGraph(cores, edges)

partition1, partition2 = kl_partitioning(graph, 0, cores)

print( "Cost:" f"{calculate_cost( graph, partition1, partition2 )}" )
print("Final Partition 1:", partition1)
print("Final Partition 2:", partition2)

Level 0:
Complete Set: {'C15', 'C4', 'C7', 'C10', 'C2', 'C1', 'C13', 'C16', 'C6', 'C14', 'C9', 'C8', 'C3', 'C11', 'C5', 'C12'}
Start Partitions Cost: 10
 Partition 1: {'C13', 'C4', 'C7', 'C16', 'C6', 'C8', 'C10', 'C5'}
 Partition 2: {'C15', 'C14', 'C9', 'C11', 'C3', 'C2', 'C1', 'C12'}
Final Partitions Cost: 4
 Partition 1: {'C4', 'C7', 'C6', 'C9', 'C8', 'C3', 'C10', 'C5'}
 Partition 2: {'C15', 'C16', 'C14', 'C11', 'C2', 'C1', 'C13', 'C12'}
    Level 1:
    Complete Set: {'C4', 'C7', 'C6', 'C9', 'C8', 'C3', 'C10', 'C5'}
    Start Partitions Cost: 4
     Partition 1: {'C4', 'C7', 'C8', 'C6'}
     Partition 2: {'C9', 'C3', 'C5', 'C10'}
    Final Partitions Cost: 4
     Partition 1: {'C4', 'C7', 'C8', 'C6'}
     Partition 2: {'C9', 'C3', 'C5', 'C10'}
        Level 2:
        Complete Set: {'C4', 'C7', 'C8', 'C6'}
        Start Partitions Cost: 2
         Partition 1: {'C4', 'C8'}
         Partition 2: {'C7', 'C6'}
        Final Partitions Cost: 0
         Partition 1: {'C7', 'C8'}
        

# Weighted Kernighan-Lin Partition

## Graph

In [ ]:
class CoreGraph:
    def __init__(self, cores, edges):
        self.cores = cores
        self.edges = edges
        self.weights = {edge: random.randint(1, 10) for edge in edges}

## Calculate Cost

In [ ]:
def calculate_cost(graph, partition1, partition2):
    cost = 0
    for edge, weight in graph.weights.items():
        if (edge[0] in partition1 and edge[1] in partition2) or (edge[0] in partition2 and edge[1] in partition1):
            cost += weight
    return cost

## Swap Core

In [ ]:
def get_next_move(graph, partition1, partition2):
    best_cost = calculate_cost( graph, partition1, partition2 )
    best_move = None
    for ci in partition1:
        for cj in partition2:
            current_cost = calculate_cost(graph, partition1 - {ci} | {cj}, partition2 - {cj} | {ci})
            if current_cost < best_cost:
                best_cost = current_cost
                best_move = (ci, cj)
    return best_move

## Kernighan_Lin Partition

In [ ]:
def kernighan_lin(graph, partition1, partition2, level=0):
    best_partition1, best_partition2 = partition1.copy(), partition2.copy()
    current_partition1, current_partition2 = partition1.copy(), partition2.copy()

    max_iterations = 50

    for iteration in range(max_iterations):
        next_move = get_next_move(graph, current_partition1, current_partition2)
        if next_move is None:
            break  # Salimos del ciclo si no encontramos una mejor jugada
        ci, cj = next_move
        current_partition1.remove(ci)
        current_partition2.remove(cj)
        current_partition1.add(cj)
        current_partition2.add(ci)

        if calculate_cost(graph, current_partition1, current_partition2) < calculate_cost(graph, best_partition1, best_partition2):
            best_partition1, best_partition2 = current_partition1.copy(), current_partition2.copy()

    if len(best_partition1) == len(partition1) and len(best_partition2) == len(partition2):
        return best_partition1, best_partition2

    p1, p2 = best_partition1, best_partition2
    return p1, p2

## Extend Kernighan-Lin Partition

In [ ]:
def kl_partitioning(graph, level, cores_to_be_partitioned):
  cores_to_be_partitioned = set(cores_to_be_partitioned)  # Convert to set to allow set operations

  if len(cores_to_be_partitioned) < 4:
    return cores_to_be_partitioned

  cores_list = list(cores_to_be_partitioned)

  partition1 = set(random.sample(cores_list, len(cores_list) // 2))
  partition2 = cores_to_be_partitioned - partition1

  indent = " " * (level * 4)
  print(indent + f"Level {level}:")
  print(indent + f"Complete Set: {cores_to_be_partitioned}")
  print(indent + f"Start Partitions" + f" Cost: {calculate_cost(graph, partition1, partition2)}")
  print(indent + f" Partition 1: {set(partition1)}")
  print(indent + f" Partition 2: {set(partition2)}")

  partition1, partition2 = kernighan_lin(graph, partition1, partition2)

  indent = " " * (level * 4)
  print(indent + f"Final Partitions" + f" Cost: {calculate_cost(graph, partition1, partition2)}")
  print(indent + f" Partition 1: {set(partition1)}")
  print(indent + f" Partition 2: {set(partition2)}")

  partition1_1, partition1_2 = kl_partitioning(graph, level + 1, partition1)
  partition2_1, partition2_2 = kl_partitioning(graph, level + 1, partition2)

  partition1 = (partition1_1, partition1_2)
  partition2 = (partition2_1, partition2_2)

  return partition1, partition2


## Test

In [ ]:
# Ejemplo de uso
cores = {'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'C10', 'C11', 'C12', 'C13', 'C14', 'C15', 'C16'}
edges = {('C1', 'C2'), ('C1', 'C3'), ('C2', 'C4'), ('C3', 'C4'), ('C3', 'C5'), ('C4', 'C6'), ('C5', 'C6'),
         ('C7', 'C8'), ('C7', 'C9'), ('C8', 'C10'), ('C9', 'C10'), ('C9', 'C11'), ('C10', 'C12'), ('C11', 'C12'),
         ('C13', 'C14'), ('C13', 'C15'), ('C14', 'C16'), ('C15', 'C16')}
graph = CoreGraph(cores, edges)

partition1, partition2 = kl_partitioning(graph, 0, cores)

print( "Cost:" f"{calculate_cost( graph, partition1, partition2 )}" )
print("Final Partition 1:", partition1)
print("Final Partition 2:", partition2)

Level 0:
Complete Set: {'C15', 'C4', 'C7', 'C10', 'C2', 'C1', 'C13', 'C16', 'C6', 'C14', 'C9', 'C8', 'C3', 'C11', 'C5', 'C12'}
Start Partitions Cost: 54
 Partition 1: {'C15', 'C4', 'C6', 'C11', 'C8', 'C3', 'C1', 'C13'}
 Partition 2: {'C7', 'C16', 'C14', 'C9', 'C10', 'C2', 'C5', 'C12'}
Final Partitions Cost: 5
 Partition 1: {'C13', 'C4', 'C6', 'C14', 'C3', 'C2', 'C1', 'C5'}
 Partition 2: {'C15', 'C7', 'C16', 'C9', 'C11', 'C8', 'C10', 'C12'}
    Level 1:
    Complete Set: {'C13', 'C4', 'C6', 'C14', 'C3', 'C2', 'C1', 'C5'}
    Start Partitions Cost: 29
     Partition 1: {'C2', 'C5', 'C3', 'C14'}
     Partition 2: {'C4', 'C1', 'C6', 'C13'}
    Final Partitions Cost: 13
     Partition 1: {'C2', 'C1', 'C3', 'C5'}
     Partition 2: {'C13', 'C4', 'C6', 'C14'}
        Level 2:
        Complete Set: {'C2', 'C1', 'C3', 'C5'}
        Start Partitions Cost: 6
         Partition 1: {'C3', 'C5'}
         Partition 2: {'C2', 'C1'}
        Final Partitions Cost: 6
         Partition 1: {'C3', 'C5'}
   

# Weighted Internal-External Kernighan-Lin Partition

## Graph

In [ ]:
import random
class CoreGraph:
    def __init__(self, cores, edges):
        self.cores = cores
        self.edges = edges
        self.weights = {edge: random.randint(1, 10) for edge in edges}

## Calculate Cost

In [ ]:
def calculate_cost(graph, partition1, partition2):
    external_cost = 0
    internal_cost = 0

    for edge, weight in graph.weights.items():
        if (edge[0] in partition1 and edge[1] in partition2) or (edge[0] in partition2 and edge[1] in partition1):
            external_cost += weight
        elif (edge[0] in partition1 and edge[1] in partition1) or (edge[0] in partition2 and edge[1] in partition2):
            internal_cost += weight

    return external_cost - internal_cost

## Swap Core

In [ ]:
def get_next_move(graph, partition1, partition2, locked_vertices):
  best_cost = calculate_cost( graph, partition1, partition2 )
  best_move = None
  for ci in partition1:
    if ci in locked_vertices:
      continue
    for cj in partition2:
      if cj in locked_vertices:
        continue
      current_cost = calculate_cost(graph, partition1 - {ci} | {cj}, partition2 - {cj} | {ci})
      if current_cost < best_cost:
        best_cost = current_cost
        best_move = (ci, cj)
  return best_move

## Kernighan_Lin Partition

In [ ]:
def kernighan_lin(graph, partition1, partition2, level=0):
    best_partition1, best_partition2 = partition1.copy(), partition2.copy()
    current_partition1, current_partition2 = partition1.copy(), partition2.copy()

    max_iterations = 5

    for iteration in range(max_iterations):
      locked_vertices = set()
      print( iteration )
      while( len(current_partition1) > 1 ):
        next_move = get_next_move(graph, current_partition1, current_partition2, locked_vertices)
        print( next_move )
        if next_move is None:
            break  # Salimos del ciclo si no encontramos una mejor jugada
        ci, cj = next_move
        current_partition1.remove(ci)
        current_partition2.remove(cj)
        current_partition1.add(cj)
        current_partition2.add(ci)
        locked_vertices.add(ci)
        locked_vertices.add(cj)

        if calculate_cost(graph, current_partition1, current_partition2) < calculate_cost(graph, best_partition1, best_partition2):
            best_partition1, best_partition2 = current_partition1.copy(), current_partition2.copy()

    if len(best_partition1) == len(partition1) and len(best_partition2) == len(partition2):
      return best_partition1, best_partition2

    p1, p2 = best_partition1, best_partition2
    return p1, p2

## Extend Kernighan-Lin Partition

In [ ]:
def kl_partitioning(graph, level, cores_to_be_partitioned):
  cores_to_be_partitioned = set(cores_to_be_partitioned)  # Convert to set to allow set operations

  if len(cores_to_be_partitioned) < 4:
    return cores_to_be_partitioned

  cores_list = list(cores_to_be_partitioned)

  partition1 = set(random.sample(cores_list, len(cores_list) // 2))
  partition2 = cores_to_be_partitioned - partition1

  indent = " " * (level * 4)
  print(indent + f"Level {level}:")
  print(indent + f"Complete Set: {cores_to_be_partitioned}")
  print(indent + f"Start Partitions" + f" Cost: {calculate_cost(graph, partition1, partition2)}")
  print(indent + f" Partition 1: {set(partition1)}")
  print(indent + f" Partition 2: {set(partition2)}")

  partition1, partition2 = kernighan_lin(graph, partition1, partition2)

  indent = " " * (level * 4)
  print(indent + f"Final Partitions" + f" Cost: {calculate_cost(graph, partition1, partition2)}")
  print(indent + f" Partition 1: {set(partition1)}")
  print(indent + f" Partition 2: {set(partition2)}")

  partition1_1, partition1_2 = kl_partitioning(graph, level + 1, partition1)
  partition2_1, partition2_2 = kl_partitioning(graph, level + 1, partition2)

  partition1 = (partition1_1, partition1_2)
  partition2 = (partition2_1, partition2_2)

  return partition1, partition2


## Test

In [ ]:
# Ejemplo de uso
cores = {'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'C10', 'C11', 'C12', 'C13', 'C14', 'C15', 'C16'}
edges = {('C1', 'C2'), ('C1', 'C3'), ('C2', 'C4'), ('C3', 'C4'), ('C3', 'C5'), ('C4', 'C6'), ('C5', 'C6'),
         ('C7', 'C8'), ('C7', 'C9'), ('C8', 'C10'), ('C9', 'C10'), ('C9', 'C11'), ('C10', 'C12'), ('C11', 'C12'),
         ('C13', 'C14'), ('C13', 'C15'), ('C14', 'C16'), ('C15', 'C16')}
graph = CoreGraph(cores, edges)

partition1, partition2 = kl_partitioning(graph, 0, cores)

print( "Cost:" f"{calculate_cost( graph, partition1, partition2 )}" )
print("Final Partition 1:", partition1)
print("Final Partition 2:", partition2)

Level 0:
Complete Set: {'C5', 'C7', 'C2', 'C14', 'C6', 'C4', 'C1', 'C3', 'C9', 'C10', 'C12', 'C11', 'C15', 'C13', 'C16', 'C8'}
Start Partitions Cost: 49
 Partition 1: {'C5', 'C7', 'C12', 'C11', 'C15', 'C2', 'C14', 'C6'}
 Partition 2: {'C4', 'C1', 'C3', 'C9', 'C10', 'C13', 'C16', 'C8'}
0
('C2', 'C13')
('C7', 'C16')
('C5', 'C9')
('C6', 'C10')
None
1
None
2
None
3
None
4
None
Final Partitions Cost: -95
 Partition 1: {'C9', 'C10', 'C12', 'C11', 'C15', 'C13', 'C14', 'C16'}
 Partition 2: {'C4', 'C1', 'C3', 'C5', 'C7', 'C2', 'C6', 'C8'}
    Level 1:
    Complete Set: {'C9', 'C10', 'C12', 'C11', 'C15', 'C13', 'C14', 'C16'}
    Start Partitions Cost: -7
     Partition 1: {'C9', 'C12', 'C15', 'C11'}
     Partition 2: {'C16', 'C14', 'C10', 'C13'}
0
('C15', 'C10')
None
1
None
2
None
3
None
4
None
    Final Partitions Cost: -55
     Partition 1: {'C9', 'C12', 'C11', 'C10'}
     Partition 2: {'C14', 'C15', 'C16', 'C13'}
        Level 2:
        Complete Set: {'C9', 'C12', 'C11', 'C10'}
        Start

# Weighted Example - Algorithm Kernighan-Lin

## Funciones

In [ ]:
import numpy as np

def create_external_adj_matrix(partition1, partition2, num_nodes):
  external_adj_matrix = np.zeros((num_nodes,num_nodes))

  for node1 in partition1:
    for node2 in partition2:
      external_adj_matrix[node1][node2] = 1
      external_adj_matrix[node2][node1] = 1

  return external_adj_matrix


def create_internal_adj_matrix( partition1, partition2, num_nodes ):

  internal_adj_matrix = np.ones((num_nodes,num_nodes))

  for node1 in partition1:
    for node2 in partition2:
      internal_adj_matrix[node1][node2] = 0
      internal_adj_matrix[node2][node1] = 0

  return internal_adj_matrix

def create_external_cost_matrix( cost_matrix, num_nodes ):

  external_adj_matrix = create_external_adj_matrix(partition1, partition2, num_nodes)
  external_cost_matrix = np.zeros((num_nodes,num_nodes))
  for i in range(num_nodes):
    for j in range(num_nodes):
      external_cost_matrix[i][j] = cost_matrix[i][j] * external_adj_matrix[i][j]

  return external_cost_matrix


def create_internal_cost_matrix( cost_matrix, num_nodes ):

  internal_adj_matrix = create_internal_adj_matrix(partition1, partition2, num_nodes)
  internal_cost_matrix = np.zeros((num_nodes,num_nodes))
  for i in range(num_nodes):
    for j in range(num_nodes):
      if i != j:
        internal_cost_matrix[i][j] = cost_matrix[i][j] * internal_adj_matrix[i][j]

  return internal_cost_matrix


def calculate_difference_cost_vector(  partition1, partition2, num_nodes ):

  external_cost_matrix = create_external_cost_matrix( cost_matrix, num_nodes )
  internal_cost_matrix = create_internal_cost_matrix( cost_matrix, num_nodes )
  difference_cost_matrix = np.zeros((num_nodes, num_nodes))
  for i in range(num_nodes):
    for j in range(num_nodes):
     difference_cost_matrix[i][j] += external_cost_matrix[i][j] - internal_cost_matrix[i][j]


  print("*-"*20)
  print( "External Cost Matrix" )
  print( external_cost_matrix )


  print("-"*40)
  print( "Internal Cost Matrix" )
  print( internal_cost_matrix )

  print("*-"*20)
  print( "\n" )

  return np.dot([1,1,1,1,1,1],difference_cost_matrix)

def update_difference_cost_vector(  difference_cost_vector, partition1, partition2, available_nodes, swap_nodes  ):

    internal_adj_matrix = create_internal_adj_matrix( partition1, partition2, num_nodes)
    difference_cost_internal_matrix = np.zeros( (num_nodes, num_nodes)  )
    for i in range(num_nodes):
      for j in range(num_nodes):
        difference_cost_internal_matrix[i][j] = cost_matrix[i][j] * internal_adj_matrix[i][j]

    external_adj_matrix = create_external_adj_matrix( partition1, partition2, num_nodes)
    difference_cost_external_matrix = np.zeros( (num_nodes, num_nodes)  )
    for i in range(num_nodes):
      for j in range(num_nodes):
          difference_cost_external_matrix[i][j] = cost_matrix[i][j] * external_adj_matrix[i][j]

    difference_cost_prime_matrix = np.zeros((num_nodes,num_nodes))
    for i in range(num_nodes):
      for j in range(num_nodes):
        difference_cost_prime_matrix[i][j] = difference_cost_internal_matrix[i][j] - difference_cost_external_matrix[i][j]

    difference_cost_vector          = np.multiply(available_nodes, difference_cost_vector)
    difference_cost_internal_vector = np.dot(swap_nodes, difference_cost_internal_matrix)
    difference_cost_external_vector = np.dot(swap_nodes, difference_cost_external_matrix)
    difference_cost_prime_vector    = difference_cost_vector + 2 * difference_cost_internal_vector - 2 * difference_cost_external_vector

    print(internal_adj_matrix)
    print("*-"*20)
    print( "Difference Cost Internal Matrix")
    for row in difference_cost_internal_matrix:
      print(row, np.sum(row))

    print("-"*40)

    print( "Difference Cost Internal Vector")
    print( difference_cost_internal_vector )
    print("*-"*20)

    print("\n")

    print("*-"*20)
    print( "Difference Cost External Matrix")
    for row in difference_cost_external_matrix:
      print(row, np.sum(row))

    print("-"*40)

    print( "Difference Cost External Vector")
    print( difference_cost_external_vector )
    print("*-"*20)

    print("\n")

    print("*-"*20)
    print( "Difference Cost Vector")
    print( difference_cost_vector)

    print("-"*40)

    print( "Difference Cost Updated Vector")
    print( difference_cost_prime_vector )

    print("*-"*20)
    print("\n")

    return difference_cost_prime_vector

def calculate_gain_cost_matrix( difference_cost_vector, external_adj_matrix, cost_matrix ):
  gain_cost_matrix = np.full((num_nodes,num_nodes), 0)

  for i in range(num_nodes):
    for j in range(num_nodes):
      if external_adj_matrix[i][j] == 1:
        gain_cost_matrix[i][j] = difference_cost_vector[i] + difference_cost_vector[j] - 2 * cost_matrix[i][j]

  return gain_cost_matrix


def swap_cores_partition( partition1, partition2, gain_cost_matrix, available_nodes ):
  partition1_copy = partition1.copy()
  partition2_copy = partition2.copy()
  max_gain = -float('inf')
  max_i, max_j = None, None

  for i in range( num_nodes ):
    for j in range( num_nodes ):
      if max_gain < gain_cost_matrix[i][j] and gain_cost_matrix[i][j] != 0 and available_nodes[i] == 1 and available_nodes[j] == 1:
        max_gain = gain_cost_matrix[i][j]
        max_i = i
        max_j = j

  print( max_i, max_j, max_gain )
  next_move = [max_gain, max_i, max_j]
  for ci in partition1:
    for cj in partition2:
      if ci == max_i and cj == max_j:
        partition1_copy.remove(ci)
        partition2_copy.remove(cj)
        partition1_copy.add(cj)
        partition2_copy.add(ci)

  return partition1_copy, partition2_copy, next_move

def max_subarray_sum(next_moves):
    select_next_moves = []

    current_sum = 0
    max_sum = 0

    for next_move in next_moves:
        current_sum += int(next_move[0])
        if current_sum > max_sum:
          max_sum = current_sum
          select_next_moves.append( next_move )

    return max_sum, select_next_moves

def swap_cores(  partial_next_moves, partition1, partition2  ):
  for next_move in partial_next_moves:
    for ci in partition1:
     for cj in partition2:
        if ci == next_move[1] and cj == next_move[2]:
          partition1.remove(ci)
          partition2.remove(cj)
          partition1.add(cj)
          partition2.add(ci)
  return partition1, partition2


## Ejemplo

In [ ]:
# Ejemplo de uso
if __name__ == "__main__":
  partition1 = {0, 1, 2}
  partition2 = {3, 4, 5}
  num_nodes = 6
  next_moves = []

  cost_matrix = [ [0,1,2,3,2,4],
                  [1,0,1,4,2,1],
                  [2,1,0,3,2,1],
                  [3,4,3,0,4,3],
                  [2,2,2,4,0,2],
                  [4,1,1,3,2,0] ]




In [ ]:
  external_adj_matrix = create_external_adj_matrix(partition1, partition2, num_nodes)
  internal_adj_matrix = create_internal_adj_matrix(partition1, partition2, num_nodes)
  #external_cost_matrix = create_external_cost_matrix( cost_matrix, external_adj_matrix, num_nodes )
  #internal_cost_matrix = create_internal_cost_matrix( cost_matrix, internal_adj_matrix, num_nodes )

  external_cost_matrix = create_external_cost_matrix( cost_matrix, num_nodes )
  internal_cost_matrix = create_internal_cost_matrix( cost_matrix, num_nodes )

### Iteracion #1

#### Intercambio #1

In [ ]:
  difference_cost_vector =  calculate_difference_cost_vector( partition1, partition2, num_nodes  )
  gain_cost_matrix       =  calculate_gain_cost_matrix( difference_cost_vector, external_adj_matrix, cost_matrix )

  available_nodes = [1,1,1,1,1,1]
  partition1_swapped, partition2_swapped, next_move = swap_cores_partition( partition1, partition2, gain_cost_matrix, available_nodes )

  next_moves.append( next_move )

  print("-"*40)
  print("Difference cost Vector")
  print( difference_cost_vector )
  print("-"*40)

  print("\n")

  print("-"*40)
  print("Gain Cost Matrix")
  print( gain_cost_matrix )
  print("-"*40)

  print("\n")

  print("-"*40)
  print("Particiones")
  print( "Partition1:", partition1_swapped )
  print( "Partition2:", partition2_swapped )

*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-
External Cost Matrix
[[0. 0. 0. 3. 2. 4.]
 [0. 0. 0. 4. 2. 1.]
 [0. 0. 0. 3. 2. 1.]
 [3. 4. 3. 0. 0. 0.]
 [2. 2. 2. 0. 0. 0.]
 [4. 1. 1. 0. 0. 0.]]
----------------------------------------
Internal Cost Matrix
[[0. 1. 2. 0. 0. 0.]
 [1. 0. 1. 0. 0. 0.]
 [2. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 4. 3.]
 [0. 0. 0. 4. 0. 2.]
 [0. 0. 0. 3. 2. 0.]]
*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-


1 5 4
----------------------------------------
Difference cost Vector
[6. 5. 3. 3. 0. 1.]
----------------------------------------


----------------------------------------
Gain Cost Matrix
[[ 0  0  0  3  2 -1]
 [ 0  0  0  0  1  4]
 [ 0  0  0  0 -1  2]
 [ 3  0  0  0  0  0]
 [ 2  1 -1  0  0  0]
 [-1  4  2  0  0  0]]
----------------------------------------


----------------------------------------
Particiones
Partition1: {0, 2, 5}
Partition2: {1, 3, 4}


#### Intercambio #2

In [ ]:
  available_nodes = [1,0,1,1,1,0]
  swap_nodes      = [0,1,0,0,0,1]

  difference_cost_prime_vector =  update_difference_cost_vector( difference_cost_vector, partition1, partition2, available_nodes, swap_nodes )
  gain_cost_matrix = calculate_gain_cost_matrix( difference_cost_prime_vector,  create_external_adj_matrix( partition1_swapped, partition2_swapped, num_nodes ), cost_matrix  )

  print(  gain_cost_matrix  )

  partition1_swapped, partition2_swapped, next_move =  swap_cores_partition( partition1_swapped, partition2_swapped, gain_cost_matrix, available_nodes )
  next_moves.append( next_move )

  print( "Partition1:", partition1_swapped )
  print( "Partition2:", partition2_swapped )

[[1. 1. 1. 0. 0. 0.]
 [1. 1. 1. 0. 0. 0.]
 [1. 1. 1. 0. 0. 0.]
 [0. 0. 0. 1. 1. 1.]
 [0. 0. 0. 1. 1. 1.]
 [0. 0. 0. 1. 1. 1.]]
*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-
Difference Cost Internal Matrix
[0. 1. 2. 0. 0. 0.] 3.0
[1. 0. 1. 0. 0. 0.] 2.0
[2. 1. 0. 0. 0. 0.] 3.0
[0. 0. 0. 0. 4. 3.] 7.0
[0. 0. 0. 4. 0. 2.] 6.0
[0. 0. 0. 3. 2. 0.] 5.0
----------------------------------------
Difference Cost Internal Vector
[1. 0. 1. 3. 2. 0.]
*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-


*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-
Difference Cost External Matrix
[0. 0. 0. 3. 2. 4.] 9.0
[0. 0. 0. 4. 2. 1.] 7.0
[0. 0. 0. 3. 2. 1.] 6.0
[3. 4. 3. 0. 0. 0.] 10.0
[2. 2. 2. 0. 0. 0.] 6.0
[4. 1. 1. 0. 0. 0.] 6.0
----------------------------------------
Difference Cost External Vector
[4. 1. 1. 4. 2. 1.]
*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-


*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-
Difference Cost Vector
[6. 0. 3. 3. 0. 0.]
----------------------------------------
Difference Cost Updated Vector
[ 0. -2.  

#### Intercambio #3

In [ ]:
  available_nodes = [1,0,0,1,0,0]
  swap_nodes = [0,0,1,0,1,0]

  difference_cost_prime_vector =  update_difference_cost_vector( difference_cost_prime_vector, partition1, partition2, available_nodes, swap_nodes )
  gain_cost_matrix = calculate_gain_cost_matrix( difference_cost_prime_vector,  create_external_adj_matrix( partition1, partition2, num_nodes ), cost_matrix  )
  print( gain_cost_matrix  )
  available_nodes = [1,0,0,1,0,0]
  partition1_swapped, partition2_swapped, next_move =  swap_cores_partition( partition1_swapped, partition2_swapped, gain_cost_matrix, available_nodes )
  next_moves.append( next_move )

[[1. 1. 1. 0. 0. 0.]
 [1. 1. 1. 0. 0. 0.]
 [1. 1. 1. 0. 0. 0.]
 [0. 0. 0. 1. 1. 1.]
 [0. 0. 0. 1. 1. 1.]
 [0. 0. 0. 1. 1. 1.]]
*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-
Difference Cost Internal Matrix
[0. 1. 2. 0. 0. 0.] 3.0
[1. 0. 1. 0. 0. 0.] 2.0
[2. 1. 0. 0. 0. 0.] 3.0
[0. 0. 0. 0. 4. 3.] 7.0
[0. 0. 0. 4. 0. 2.] 6.0
[0. 0. 0. 3. 2. 0.] 5.0
----------------------------------------
Difference Cost Internal Vector
[2. 1. 0. 4. 0. 2.]
*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-


*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-
Difference Cost External Matrix
[0. 0. 0. 3. 2. 4.] 9.0
[0. 0. 0. 4. 2. 1.] 7.0
[0. 0. 0. 3. 2. 1.] 6.0
[3. 4. 3. 0. 0. 0.] 10.0
[2. 2. 2. 0. 0. 0.] 6.0
[4. 1. 1. 0. 0. 0.] 6.0
----------------------------------------
Difference Cost External Vector
[2. 2. 2. 3. 2. 1.]
*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-


*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-
Difference Cost Vector
[ 0. -0.  0.  1.  0. -0.]
----------------------------------------
Difference Cost Updated Vector
[ 0.

#### Movimientos Parciales

In [ ]:
  max_partial_sum, partial_next_moves = max_subarray_sum( next_moves )
  print(max_partial_sum, partial_next_moves)

4 [[4, 1, 5]]


#### Intercambio

In [ ]:
  partition1, partition2 = swap_cores( partial_next_moves, partition1, partition2 )
  print( partition1 )
  print( partition2 )

{0, 2, 5}
{1, 3, 4}


### Iteracion #2

In [ ]:
  external_adj_matrix = create_external_adj_matrix(partition1, partition2, num_nodes)
  internal_adj_matrix = create_internal_adj_matrix(partition1, partition2, num_nodes)
  #external_cost_matrix = create_external_cost_matrix( cost_matrix, external_adj_matrix, num_nodes )
  #internal_cost_matrix = create_internal_cost_matrix( cost_matrix, internal_adj_matrix, num_nodes )

  external_cost_matrix = create_external_cost_matrix( cost_matrix, num_nodes )
  internal_cost_matrix = create_internal_cost_matrix( cost_matrix, num_nodes )
  next_moves = []

#### Intercambio #1

In [ ]:
  difference_cost_vector =  calculate_difference_cost_vector( partition1, partition2, num_nodes  )
  gain_cost_matrix       =  calculate_gain_cost_matrix( difference_cost_vector, external_adj_matrix, cost_matrix )

  available_nodes = [1,1,1,1,1,1]
  partition1_swapped, partition2_swapped, next_move = swap_cores_partition( partition1, partition2, gain_cost_matrix, available_nodes )

  next_moves.append( next_move )

  print("-"*40)
  print("Difference cost Vector")
  print( difference_cost_vector )
  print("-"*40)

  print("\n")

  print("-"*40)
  print("Gain Cost Matrix")
  print( gain_cost_matrix )
  print("-"*40)

  print("\n")

  print("-"*40)
  print("Particiones")
  print( "Partition1:", partition1_swapped )
  print( "Partition2:", partition2_swapped )

*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-
External Cost Matrix
[[0. 1. 0. 3. 2. 0.]
 [1. 0. 1. 0. 0. 1.]
 [0. 1. 0. 3. 2. 0.]
 [3. 0. 3. 0. 0. 3.]
 [2. 0. 2. 0. 0. 2.]
 [0. 1. 0. 3. 2. 0.]]
----------------------------------------
Internal Cost Matrix
[[0. 0. 2. 0. 0. 4.]
 [0. 0. 0. 4. 2. 0.]
 [2. 0. 0. 0. 0. 1.]
 [0. 4. 0. 0. 4. 0.]
 [0. 2. 0. 4. 0. 0.]
 [4. 0. 1. 0. 0. 0.]]
*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-


2 4 -1
----------------------------------------
Difference cost Vector
[ 0. -3.  3.  1.  0.  1.]
----------------------------------------


----------------------------------------
Gain Cost Matrix
[[ 0 -5  0 -5 -4  0]
 [-5  0 -2  0  0 -4]
 [ 0 -2  0 -2 -1  0]
 [-5  0 -2  0  0 -4]
 [-4  0 -1  0  0 -3]
 [ 0 -4  0 -4 -3  0]]
----------------------------------------


----------------------------------------
Particiones
Partition1: {0, 4, 5}
Partition2: {1, 2, 3}


#### Intercambio #2

In [ ]:
  available_nodes = [1,1,0,1,0,1]
  swap_nodes      = [0,0,1,0,1,0]

  difference_cost_prime_vector =  update_difference_cost_vector( difference_cost_vector, partition1, partition2, available_nodes, swap_nodes )
  gain_cost_matrix = calculate_gain_cost_matrix( difference_cost_prime_vector,  create_external_adj_matrix( partition1_swapped, partition2_swapped, num_nodes ), cost_matrix  )

  print(  gain_cost_matrix  )

  partition1_swapped, partition2_swapped, next_move =  swap_cores_partition( partition1_swapped, partition2_swapped, gain_cost_matrix, available_nodes )
  next_moves.append( next_move )

  print( "Partition1:", partition1_swapped )
  print( "Partition2:", partition2_swapped )

[[1. 0. 1. 0. 0. 1.]
 [0. 1. 0. 1. 1. 0.]
 [1. 0. 1. 0. 0. 1.]
 [0. 1. 0. 1. 1. 0.]
 [0. 1. 0. 1. 1. 0.]
 [1. 0. 1. 0. 0. 1.]]
*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-
Difference Cost Internal Matrix
[0. 0. 2. 0. 0. 4.] 6.0
[0. 0. 0. 4. 2. 0.] 6.0
[2. 0. 0. 0. 0. 1.] 3.0
[0. 4. 0. 0. 4. 0.] 8.0
[0. 2. 0. 4. 0. 0.] 6.0
[4. 0. 1. 0. 0. 0.] 5.0
----------------------------------------
Difference Cost Internal Vector
[2. 2. 0. 4. 0. 1.]
*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-


*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-
Difference Cost External Matrix
[0. 1. 0. 3. 2. 0.] 6.0
[1. 0. 1. 0. 0. 1.] 3.0
[0. 1. 0. 3. 2. 0.] 6.0
[3. 0. 3. 0. 0. 3.] 9.0
[2. 0. 2. 0. 0. 2.] 6.0
[0. 1. 0. 3. 2. 0.] 6.0
----------------------------------------
Difference Cost External Vector
[2. 1. 2. 3. 2. 2.]
*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-


*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-
Difference Cost Vector
[ 0. -3.  0.  1.  0.  1.]
----------------------------------------
Difference Cost Updated Vector
[ 0. 

#### Intercambio #3

In [ ]:
  available_nodes = [0,0,0,1,0,1]
  swap_nodes      = [1,1,0,0,0,0]

  difference_cost_prime_vector =  update_difference_cost_vector( difference_cost_prime_vector, partition1, partition2, available_nodes, swap_nodes )
  gain_cost_matrix = calculate_gain_cost_matrix( difference_cost_prime_vector,  create_external_adj_matrix( partition1, partition2, num_nodes ), cost_matrix  )
  print( gain_cost_matrix  )

  partition1_swapped, partition2_swapped, next_move =  swap_cores_partition( partition1_swapped, partition2_swapped, gain_cost_matrix, available_nodes )
  next_moves.append( next_move )

[[1. 0. 1. 0. 0. 1.]
 [0. 1. 0. 1. 1. 0.]
 [1. 0. 1. 0. 0. 1.]
 [0. 1. 0. 1. 1. 0.]
 [0. 1. 0. 1. 1. 0.]
 [1. 0. 1. 0. 0. 1.]]
*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-
Difference Cost Internal Matrix
[0. 0. 2. 0. 0. 4.] 6.0
[0. 0. 0. 4. 2. 0.] 6.0
[2. 0. 0. 0. 0. 1.] 3.0
[0. 4. 0. 0. 4. 0.] 8.0
[0. 2. 0. 4. 0. 0.] 6.0
[4. 0. 1. 0. 0. 0.] 5.0
----------------------------------------
Difference Cost Internal Vector
[0. 0. 2. 4. 2. 4.]
*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-


*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-
Difference Cost External Matrix
[0. 1. 0. 3. 2. 0.] 6.0
[1. 0. 1. 0. 0. 1.] 3.0
[0. 1. 0. 3. 2. 0.] 6.0
[3. 0. 3. 0. 0. 3.] 9.0
[2. 0. 2. 0. 0. 2.] 6.0
[0. 1. 0. 3. 2. 0.] 6.0
----------------------------------------
Difference Cost External Vector
[1. 1. 1. 3. 2. 1.]
*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-


*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-
Difference Cost Vector
[ 0. -0. -0.  3. -0. -1.]
----------------------------------------
Difference Cost Updated Vector
[-2. 

#### Movimientos Parciales

In [ ]:
  max_partial_sum, partial_next_moves = max_subarray_sum( next_moves )

  print(max_partial_sum, partial_next_moves)
  print( partial_next_moves  )

0 []
[]


#### Intercambio

In [ ]:
partition1, partition2 = swap_cores( partial_next_moves, partition1, partition2 )
print( partition1 )
print( partition2 )

{0, 2, 5}
{1, 3, 4}


# Implementación Algoritmo Kernighan-Lin

# Bibliotecas

In [128]:
import numpy as np

## Funciones

In [4]:
import numpy as np

def create_external_adj_matrix(partition1, partition2, num_nodes):
  external_adj_matrix = np.zeros((num_nodes,num_nodes))

  for node1 in partition1:
    for node2 in partition2:
      external_adj_matrix[node1][node2] = 1
      external_adj_matrix[node2][node1] = 1

  return external_adj_matrix


def create_internal_adj_matrix( partition1, partition2, num_nodes ):

  internal_adj_matrix = np.ones((num_nodes,num_nodes))

  for node1 in partition1:
    for node2 in partition2:
      internal_adj_matrix[node1][node2] = 0
      internal_adj_matrix[node2][node1] = 0

  return internal_adj_matrix

def create_external_cost_matrix( cost_matrix, num_nodes ):

  external_adj_matrix = create_external_adj_matrix(partition1, partition2, num_nodes)
  external_cost_matrix = np.zeros((num_nodes,num_nodes))
  for i in range(num_nodes):
    for j in range(num_nodes):
      external_cost_matrix[i][j] = cost_matrix[i][j] * external_adj_matrix[i][j]

  return external_cost_matrix


def create_internal_cost_matrix( cost_matrix, num_nodes ):

  internal_adj_matrix = create_internal_adj_matrix(partition1, partition2, num_nodes)
  internal_cost_matrix = np.zeros((num_nodes,num_nodes))
  for i in range(num_nodes):
    for j in range(num_nodes):
      if i != j:
        internal_cost_matrix[i][j] = cost_matrix[i][j] * internal_adj_matrix[i][j]

  return internal_cost_matrix


def calculate_difference_cost_vector(  partition1, partition2, num_nodes ):

  external_cost_matrix = create_external_cost_matrix( cost_matrix, num_nodes )
  internal_cost_matrix = create_internal_cost_matrix( cost_matrix, num_nodes )
  difference_cost_matrix = np.zeros((num_nodes, num_nodes))
  for i in range(num_nodes):
    for j in range(num_nodes):
     difference_cost_matrix[i][j] += external_cost_matrix[i][j] - internal_cost_matrix[i][j]


  print("*-"*20)
  print( "External Cost Matrix" )
  print( external_cost_matrix )


  print("-"*40)
  print( "Internal Cost Matrix" )
  print( internal_cost_matrix )

  print("*-"*20)
  print( "\n" )

  return np.dot([1,1,1,1,1,1],difference_cost_matrix)

def update_difference_cost_vector(  difference_cost_vector, partition1, partition2, available_nodes, swap_nodes  ):

    internal_adj_matrix = create_internal_adj_matrix( partition1, partition2, num_nodes)
    difference_cost_internal_matrix = np.zeros( (num_nodes, num_nodes)  )
    for i in range(num_nodes):
      for j in range(num_nodes):
        difference_cost_internal_matrix[i][j] = cost_matrix[i][j] * internal_adj_matrix[i][j]

    external_adj_matrix = create_external_adj_matrix( partition1, partition2, num_nodes)
    difference_cost_external_matrix = np.zeros( (num_nodes, num_nodes)  )
    for i in range(num_nodes):
      for j in range(num_nodes):
          difference_cost_external_matrix[i][j] = cost_matrix[i][j] * external_adj_matrix[i][j]

    difference_cost_prime_matrix = np.zeros((num_nodes,num_nodes))
    for i in range(num_nodes):
      for j in range(num_nodes):
        difference_cost_prime_matrix[i][j] = difference_cost_internal_matrix[i][j] - difference_cost_external_matrix[i][j]

    difference_cost_vector          = np.multiply(available_nodes, difference_cost_vector)
    difference_cost_internal_vector = np.dot(swap_nodes, difference_cost_internal_matrix)
    difference_cost_external_vector = np.dot(swap_nodes, difference_cost_external_matrix)
    difference_cost_prime_vector    = difference_cost_vector + 2 * difference_cost_internal_vector - 2 * difference_cost_external_vector

    print(internal_adj_matrix)
    print("*-"*20)
    print( "Difference Cost Internal Matrix")
    for row in difference_cost_internal_matrix:
      print(row, np.sum(row))

    print("-"*40)

    print( "Difference Cost Internal Vector")
    print( difference_cost_internal_vector )
    print("*-"*20)

    print("\n")

    print("*-"*20)
    print( "Difference Cost External Matrix")
    for row in difference_cost_external_matrix:
      print(row, np.sum(row))

    print("-"*40)

    print( "Difference Cost External Vector")
    print( difference_cost_external_vector )
    print("*-"*20)

    print("\n")

    print("*-"*20)
    print( "Difference Cost Vector")
    print( difference_cost_vector)

    print("-"*40)

    print( "Difference Cost Updated Vector")
    print( difference_cost_prime_vector )

    print("*-"*20)
    print("\n")

    return difference_cost_prime_vector

def calculate_gain_cost_matrix( difference_cost_vector, external_adj_matrix, cost_matrix ):
  gain_cost_matrix = np.full((num_nodes,num_nodes), 0)

  for i in range(num_nodes):
    for j in range(num_nodes):
      if external_adj_matrix[i][j] == 1:
        gain_cost_matrix[i][j] = difference_cost_vector[i] + difference_cost_vector[j] - 2 * cost_matrix[i][j]

  return gain_cost_matrix


def swap_cores_partition( partition1, partition2, gain_cost_matrix, available_nodes ):
  partition1_copy = partition1.copy()
  partition2_copy = partition2.copy()
  max_gain = -float('inf')
  max_i, max_j = None, None

  for i in range( num_nodes ):
    for j in range( num_nodes ):
      if max_gain < gain_cost_matrix[i][j] and gain_cost_matrix[i][j] != 0 and available_nodes[i] == 1 and available_nodes[j] == 1:
        max_gain = gain_cost_matrix[i][j]
        max_i = i
        max_j = j

  print( max_i, max_j, max_gain )
  next_move = [max_gain, max_i, max_j]
  for ci in partition1:
    for cj in partition2:
      if ci == max_i and cj == max_j:
        partition1_copy.remove(ci)
        partition2_copy.remove(cj)
        partition1_copy.add(cj)
        partition2_copy.add(ci)

  return partition1_copy, partition2_copy, next_move

def max_subarray_sum(next_moves):
    select_next_moves = []

    current_sum = 0
    max_sum = 0

    for next_move in next_moves:
        current_sum += int(next_move[0])
        if current_sum > max_sum:
          max_sum = current_sum
          select_next_moves.append( next_move )

    return max_sum, select_next_moves

def swap_cores(  partial_next_moves, partition1, partition2  ):
  for next_move in partial_next_moves:
    for ci in partition1:
     for cj in partition2:
        if ci == next_move[1] and cj == next_move[2]:
          partition1.remove(ci)
          partition2.remove(cj)
          partition1.add(cj)
          partition2.add(ci)
  return partition1, partition2

def update_available_nodes( available_nodes, swap_nodes ):
  return  [a ^ b for a, b in zip(available_nodes, swap_nodes)]

def update_swap_nodes( next_move ):
  indices = next_move[1:3]
  swap_nodes = [1 if i in indices else 0 for i in range(num_nodes)]
  return swap_nodes


## Ejemplo

# Iteración # 1

In [239]:
  def kernighan_lin( partition1, partition2, cost_matrix ):
    next_moves = []

    number_of_iteration   = 0
    number_of_interchange = 0

    if number_of_iteration == 0:
      print( "### Iteración 0 ###"  )
      number_of_interchange = 0
      print( "  # Intercambio 0 #" )

      if number_of_interchange == 0:
        difference_cost_vector =  calculate_difference_cost_vector( partition1, partition2, num_nodes  )
        gain_cost_matrix       =  calculate_gain_cost_matrix( difference_cost_vector, external_adj_matrix, cost_matrix )

        available_nodes = [1,1,1,1,1,1]
        partition1_swapped, partition2_swapped, next_move = swap_cores_partition( partition1, partition2, gain_cost_matrix, available_nodes )

        next_moves.append( next_move )

        print("-"*40)
        print("Difference cost Vector")
        print( difference_cost_vector )
        print("-"*40)

        print("\n")

        print("-"*40)
        print("Gain Cost Matrix")
        print( gain_cost_matrix )
        print("-"*40)

        print("\n")

        print("-"*40)
        print("Particiones")
        print( "Partition1:", partition1_swapped )
        print( "Partition2:", partition2_swapped )

        number_of_interchange = 1

      if number_of_interchange == 1:
        print( "  # Intercambio 1 #" )
        #available_nodes = [1,0,1,1,1,0]
        swap_nodes      = [0,1,0,0,0,1]


        swap_nodes      = update_swap_nodes( next_move )
        available_nodes = update_available_nodes( available_nodes, swap_nodes )

        difference_cost_prime_vector =  update_difference_cost_vector( difference_cost_vector, partition1, partition2, available_nodes, swap_nodes )
        gain_cost_matrix = calculate_gain_cost_matrix( difference_cost_prime_vector,  create_external_adj_matrix( partition1_swapped, partition2_swapped, num_nodes ), cost_matrix  )

        print(  gain_cost_matrix  )

      partition1_swapped, partition2_swapped, next_move =  swap_cores_partition( partition1_swapped, partition2_swapped, gain_cost_matrix, available_nodes )
      next_moves.append( next_move )

      print( "Partition1:", partition1_swapped )
      print( "Partition2:", partition2_swapped )

      number_of_interchange = 2

      if number_of_interchange == 2:
        print( "  # Intercambio 2 #" )
        #available_nodes = [1,0,0,1,0,0]
        swap_nodes = [0,0,1,0,1,0]

        swap_nodes      = update_swap_nodes( next_move )
        available_nodes = update_available_nodes( available_nodes, swap_nodes )

        difference_cost_prime_vector =  update_difference_cost_vector( difference_cost_prime_vector, partition1, partition2, available_nodes, swap_nodes )
        gain_cost_matrix = calculate_gain_cost_matrix( difference_cost_prime_vector,  create_external_adj_matrix( partition1, partition2, num_nodes ), cost_matrix  )
        print( gain_cost_matrix  )

        partition1_swapped, partition2_swapped, next_move =  swap_cores_partition( partition1_swapped, partition2_swapped, gain_cost_matrix, available_nodes )
        next_moves.append( next_move )

      max_partial_sum, partial_next_moves = max_subarray_sum( next_moves )
      print(max_partial_sum, partial_next_moves)

      partition1, partition2 = swap_cores( partial_next_moves, partition1, partition2 )
      print( partition1 )
      print( partition2 )

      number_of_iteration = 1

    if number_of_iteration == 1:
      print( "### Iteración 1 ###"  )
      print( partition1 )
      print( partition2 )

      external_adj_matrix_1 = create_external_adj_matrix(partition1, partition2, num_nodes)
      internal_adj_matrix_1 = create_internal_adj_matrix(partition1, partition2, num_nodes)

      external_cost_matrix_1 = create_external_cost_matrix( cost_matrix, num_nodes )
      internal_cost_matrix_1 = create_internal_cost_matrix( cost_matrix, num_nodes )
      number_of_interchange = 0

      print( "# Intercambio 0 #" )
      if number_of_interchange == 0:

        difference_cost_vector =  calculate_difference_cost_vector( partition1, partition2, num_nodes  )
        gain_cost_matrix       =  calculate_gain_cost_matrix( difference_cost_vector, external_adj_matrix_1, cost_matrix )

        available_nodes = [1,1,1,1,1,1]
        partition1_swapped, partition2_swapped, next_move = swap_cores_partition( partition1, partition2, gain_cost_matrix, available_nodes )

        next_moves.append( next_move )

        print("-"*40)
        print("Difference cost Vector")
        print( difference_cost_vector )
        print("-"*40)

        print("\n")

        print("-"*40)
        print("Gain Cost Matrix")
        print( gain_cost_matrix )
        print("-"*40)

        print("\n")

        print("-"*40)
        print("Particiones")
        print( "Partition1:", partition1_swapped )
        print( "Partition2:", partition2_swapped )

        number_of_interchange = 1

      if number_of_interchange == 1:
        print( "# Intercambio 1 #" )

        #available_nodes = [1,1,0,1,0,1]
        swap_nodes      = [0,0,1,0,1,0]

        swap_nodes      = update_swap_nodes( next_move )
        available_nodes = update_available_nodes( available_nodes, swap_nodes )

        difference_cost_prime_vector =  update_difference_cost_vector( difference_cost_vector, partition1, partition2, available_nodes, swap_nodes )
        gain_cost_matrix = calculate_gain_cost_matrix( difference_cost_prime_vector,  create_external_adj_matrix( partition1_swapped, partition2_swapped, num_nodes ), cost_matrix  )

        print(  gain_cost_matrix  )

        partition1_swapped, partition2_swapped, next_move =  swap_cores_partition( partition1_swapped, partition2_swapped, gain_cost_matrix, available_nodes )
        next_moves.append( next_move )

        print( "Partition1:", partition1_swapped )
        print( "Partition2:", partition2_swapped )
        number_of_interchange = 2

      if number_of_interchange == 2:
        print( "# Intercambio 2 #" )
        #available_nodes = [0,0,0,1,0,1]
        swap_nodes      = [1,1,0,0,0,0]


        swap_nodes      = update_swap_nodes( next_move )
        available_nodes = update_available_nodes( available_nodes, swap_nodes )

        print( next_move  )
        print( available_nodes )
        print( swap_nodes )

        difference_cost_prime_vector =  update_difference_cost_vector( difference_cost_prime_vector, partition1, partition2, available_nodes, swap_nodes )
        gain_cost_matrix = calculate_gain_cost_matrix( difference_cost_prime_vector,  create_external_adj_matrix( partition1, partition2, num_nodes ), cost_matrix  )
        print( gain_cost_matrix  )

        partition1_swapped, partition2_swapped, next_move =  swap_cores_partition( partition1_swapped, partition2_swapped, gain_cost_matrix, available_nodes )
        next_moves.append( next_move )

    max_partial_sum, partial_next_moves = max_subarray_sum( next_moves )
    print(max_partial_sum, partial_next_moves)
    print( partial_next_moves  )


    partition1, partition2 = swap_cores( partial_next_moves, partition1, partition2 )
    print( partition1 )
    print( partition2 )




# Iteracion #*2*

In [1]:
def kernighan_lin( partition1, partition2, cost_matrix ):
    next_moves = []

    number_of_iteration   = 0
    number_of_interchange = 0

    for number_of_iteration in range(2):
    #if number_of_iteration == 0:

      print( "### Iteración 0 ###"  )

      available_nodes = [1,1,1,1,1,1]
      external_adj_matrix = create_external_adj_matrix(partition1, partition2, num_nodes)
      internal_adj_matrix = create_internal_adj_matrix(partition1, partition2, num_nodes)
      external_cost_matrix = create_external_cost_matrix( cost_matrix, num_nodes )
      internal_cost_matrix = create_internal_cost_matrix( cost_matrix, num_nodes )

      number_of_interchange = 0

      if number_of_interchange == 0:
        print( "  # Intercambio #" + f"{number_of_interchange}" )
        difference_cost_vector =  calculate_difference_cost_vector( partition1, partition2, num_nodes  )
        gain_cost_matrix       =  calculate_gain_cost_matrix( difference_cost_vector, external_adj_matrix, cost_matrix )
        partition1_swapped, partition2_swapped, next_move = swap_cores_partition( partition1, partition2, gain_cost_matrix, available_nodes )
        next_moves.append( next_move )

        print("-"*40)
        print("Difference cost Vector")
        print( difference_cost_vector )
        print("-"*40)

        print("\n")

        print("-"*40)
        print("Gain Cost Matrix")
        print( gain_cost_matrix )
        print("-"*40)

        print("\n")

        print("-"*40)
        print("Particiones")
        print( "Partition1:", partition1_swapped )
        print( "Partition2:", partition2_swapped )

        swap_nodes      = update_swap_nodes( next_move )
        available_nodes = update_available_nodes( available_nodes, swap_nodes )

        number_of_interchange = 1

      while np.sum(available_nodes):
        print( "  # Intercambio #" + f"{number_of_interchange}" )

        difference_cost_vector =  update_difference_cost_vector( difference_cost_vector, partition1, partition2, available_nodes, swap_nodes )
        gain_cost_matrix = calculate_gain_cost_matrix( difference_cost_vector,  create_external_adj_matrix( partition1_swapped, partition2_swapped, num_nodes ), cost_matrix  )
        partition1_swapped, partition2_swapped, next_move =  swap_cores_partition( partition1_swapped, partition2_swapped, gain_cost_matrix, available_nodes )
        next_moves.append( next_move )

        print("-"*40)
        print("Difference cost Vector")
        print( difference_cost_vector )
        print("-"*40)

        print("\n")

        print("-"*40)
        print("Gain Cost Matrix")
        print( gain_cost_matrix )
        print("-"*40)

        print("\n")

        print("-"*40)
        print("Particiones")
        print( "Partition1:", partition1_swapped )
        print( "Partition2:", partition2_swapped )

        swap_nodes      = update_swap_nodes( next_move )
        available_nodes = update_available_nodes( available_nodes, swap_nodes )

        number_of_interchange += 1

      max_partial_sum, partial_next_moves = max_subarray_sum( next_moves )
      print(max_partial_sum, partial_next_moves)

      partition1, partition2 = swap_cores( partial_next_moves, partition1, partition2 )
      print( partition1 )
      print( partition2 )



# Ejemplo

In [5]:
# Ejemplo de uso
if __name__ == "__main__":
  partition1 = {0, 1, 2}
  partition2 = {3, 4, 5}
  num_nodes = 6


  cost_matrix = [ [0,1,2,3,2,4],
                  [1,0,1,4,2,1],
                  [2,1,0,3,2,1],
                  [3,4,3,0,4,3],
                  [2,2,2,4,0,2],
                  [4,1,1,3,2,0] ]

In [6]:
kernighan_lin( partition1, partition2, cost_matrix )

### Iteración 0 ###
  # Intercambio #0
*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-
External Cost Matrix
[[0. 0. 0. 3. 2. 4.]
 [0. 0. 0. 4. 2. 1.]
 [0. 0. 0. 3. 2. 1.]
 [3. 4. 3. 0. 0. 0.]
 [2. 2. 2. 0. 0. 0.]
 [4. 1. 1. 0. 0. 0.]]
----------------------------------------
Internal Cost Matrix
[[0. 1. 2. 0. 0. 0.]
 [1. 0. 1. 0. 0. 0.]
 [2. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 4. 3.]
 [0. 0. 0. 4. 0. 2.]
 [0. 0. 0. 3. 2. 0.]]
*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-


1 5 4
----------------------------------------
Difference cost Vector
[6. 5. 3. 3. 0. 1.]
----------------------------------------


----------------------------------------
Gain Cost Matrix
[[ 0  0  0  3  2 -1]
 [ 0  0  0  0  1  4]
 [ 0  0  0  0 -1  2]
 [ 3  0  0  0  0  0]
 [ 2  1 -1  0  0  0]
 [-1  4  2  0  0  0]]
----------------------------------------


----------------------------------------
Particiones
Partition1: {0, 2, 5}
Partition2: {1, 3, 4}
  # Intercambio #1
[[1. 1. 1. 0. 0. 0.]
 [1. 1. 1. 0. 0. 0.]
 [1. 1. 1. 0. 0. 0.]
